In [2]:
from mylib.statistic_test import *

code_id = "0873 - Decision Points and Behavior"
loc = join(figpath, "Dsp", code_id)
mkdir(loc)

        D:\Data\FinalResults\Dsp\0873 - Decision Points and Behavior is made up successfully!


True

In [ ]:
DP = DPs